# Adapter Design Example JS

In [ ]:
import fs from 'fs';
import { GetObjectCommand, S3Client } from "@aws-sdk/client-s3";

interface FileSystem {
    open:(filePath: string) => void;
}

class FileSystemAdapter {
    adapter: FileSystem;

    constructor(adapter: FileSystem) {
        this.adapter = adapter;
    }

    open(path: string) {
        this.adapter.open(path);
    }
}

class LocalFileSystem implements FileSystem {
    open(path: string) {
        fs.open(path, () => {});
    }
}

class S3FileSystem implements FileSystem {
    private client: any;

    constructor() {
        this.client = new S3Client({});
    }

    async open(path: string) {
        const command = new GetObjectCommand({
            Bucket: "test-bucket",
            Key: path
        });
        const response = await this.client.send(command);
        return await response.Body.transformToString();
    }
}


// main
const fileSystem = new FileSystemAdapter(new S3FileSystem());
fileSystem.open("text.csv");

async function main() {
    // Excel csv.
    // s3 
    const path = "12345/text.csv";
    const adapter = new S3FileSystem();

    const fileSystem = new FileSystemAdapter(adapter);
    fileSystem.open("path");
}

main();


interface SocialMedia {
    getPosts: (username: string) => void;
}

class SocialMediaAdapter {
    private adapter: SocialMedia;

    constructor(adapter: SocialMedia) {
        this.adapter = adapter;
    }

    getPosts(username: string) {
        this.adapter.getPosts(username);
    }
}

class TwitterAdapter implements SocialMedia {
    getPosts(username: string) {}
}

class FacebookAdapter implements SocialMedia {
    getPosts(username: string) {}
}

const socialMedia = new SocialMediaAdapter(new TwitterAdapter())
socialMedia.getPosts("aliyagiz");

# Python equivalent

In [ ]:
import os
from typing import Dict, Any

from botocore.client import BaseClient
from botocore.exceptions import ClientError
from boto3 import client


class FileSystem:
    def open(self, file_path: str) -> None:
        raise NotImplementedError


class FileSystemAdapter:
    def __init__(self, adapter: FileSystem):
        self.adapter = adapter

    def open(self, path: str) -> None:
        self.adapter.open(path)


class LocalFileSystem(FileSystem):
    def open(self, path: str) -> None:
        os.open(path, os.O_RDWR)


class S3FileSystem(FileSystem):
    def __init__(self, client: BaseClient):
        self.client = client

    def open(self, path: str) -> str:
        try:
            response = self.client.get_object(Bucket="test-bucket", Key=path)
            return response["Body"].read().decode("utf-8")
        except ClientError as e:
            print(e)
            return ""


# main
def main():
    # Excel csv.
    # s3
    path = "12345/text.csv"
    adapter = S3FileSystem(client("s3"))

    file_system = FileSystemAdapter(adapter)
    file_system.open(path)


if __name__ == '__main__':
    main()


class SocialMedia:
    def get_posts(self, username: str) -> None:
        raise NotImplementedError


class SocialMediaAdapter:
    def __init__(self, adapter: SocialMedia):
        self.adapter = adapter

    def get_posts(self, username: str) -> None:
        self.adapter.get_posts(username)


class TwitterAdapter(SocialMedia):
    def get_posts(self, username: str) -> None:
        pass


class FacebookAdapter(SocialMedia):
    def get_posts(self, username: str) -> None:
        pass


social_media = SocialMediaAdapter(TwitterAdapter())
social_media.get_posts("aliyagiz")